# 演習3-1 単回帰分析（都心マンション価格）

In [ ]:
### 必要なパッケージ（ライブラリ）の読み込み ###
import pandas as pd             # データフレーム型変数を取り扱えるパッケージ"pandas"を読込み（以降"pd"と略記）
import matplotlib.pyplot as plt # グラフ描画のパッケージ"matplotlib"を読込み（以降"plt"と略記）
import seaborn as sns           # 上記matplotlibをベースにした高機能可視化モジュール"saeborn"を読込み（以降"sns"と略記）
import numpy as np              # 数値演算のためのパッケージ"numpy"を読込み（以降"np"と略記）

## データの理解（観察）

### データ読み込み

In [ ]:
### データの読み込み ###
df = pd.read_excel('ファイル名入力（Tabキーによる文字補完を活用）') # 都内のある区で取引された中古マンション価格データ

### データ概観把握

In [ ]:
### データそのもの ###
df

In [ ]:
### 要約統計量 ###
df.describe()

In [ ]:
### 欠測値確認 ###
df.isnull().sum()      # 欠損値件数の出力 ※df.isnull()で各セルのデータが欠損値かどうかを判定

### データ可視化（グラフ）

#### ヒストグラム

In [ ]:
df.hist()

#### 散布図

In [ ]:
sns.jointplot(data=df, x='Age', y='Price')

#### 散布図行列

In [ ]:
### 散布図行列による傾向確認 ###
sns.pairplot(df)

# 各カラム名の説明
# ID:      マンションごとの識別番号
# Price:   マンション価格（単位：
# Minutes:  駅徒歩分数
# Age:     築年数
# Area:    占有面積
# One room: ワンルームかどうかのフラグ（1:ワンルーム、0:その他間取り）

### 参考：描画設定
※重なりなどの描画崩れや、文字化けなど、描画がうまくいかない場合

In [ ]:
### 以下、デザイン設定 ###
plt.rcParams['figure.figsize'] = 10, 5             # グラフのサイズ指定
plt.rcParams['font.sans-serif'] = ['Hiragino Maru Gothic Pro', 'Yu Gothic', 'Meirio', 'Takao',
                                   'IPAexGothic', 'IPAPGothic', 'VL PGothic', 'Noto Sans CJK JP']
                                                   # 文字化け対策にフォント指定
plt.rcParams['font.size'] = 16                     # フォントサイズ一括指定
plt.tight_layout()                                 # グラフ同士が重ならないようにする
# plt.grid(True)                                   # グリッド表示ON
plt.show()                                         # 各種設定の反映

### 相関行列

In [ ]:
### 相関行列による変数間の関係性確認 ###
df.corr()  # 相関行列の算出

# pandasのv2.0.0以降、数値以外のデータが入っているとエラーになるため、
# 数値以外を含む場合、数値データのみに絞り込むオプション numeric_only=True を指定する
# （今回のデータは数値のみなので、オプション無しでもエラーなく実行できる）

In [ ]:
### 相関行列のお化粧（ヒートマップ） ###

colormap = plt.cm.RdBu_r                                   # カラーマップの設定 (RdBu:赤〜青 ⇔ RdBu_r:青〜赤)

plt.rcParams['figure.figsize'] = 10, 5                    # Figureサイズの指定

# # 相関行列の右上半分を隠したい場合は、次の2行のコメントを解除する
# mask = np.zeros_like(df.corr(), dtype=np.bool)
# mask[np.triu_indices_from(mask)] = True

sns.heatmap(df.corr(),linewidths=0.1, linecolor='white',   # 相関行列df.corr()を引数としヒートマップ作成	
            vmax=1.0, vmin=-1.0, cmap=colormap, annot=True )

            # linewidths/linecolor: 格子線の太さ/色
            # cmap:                 カラーマップの指定
            # vmax/vmin:            最大値/最小値
            # annot:                各要素への数値表示
            # mask:                 可視化から除外する対象（例：mask=mask）
            

## データの準備（加工）

In [ ]:
### N/A ###

## モデルの構築

In [ ]:
### 単回帰分析 ###
from sklearn import linear_model # 機械学習パッケージに含まれた回帰分析モジュール"linear_model"を読込

# 説明変数のセット
X = df[ ['Area'] ]

# 目的変数のセット
Y = df['Price']

# 単回帰モデルを作成
model = linear_model.LinearRegression()  # インスタンス化（関数を使える状態にする）
model.fit(X, Y)                          # モデル構築（フィッティング）

# 偏回帰係数の出力
display( pd.DataFrame({"Variables":X.columns, "Coefficients":model.coef_}) )

# 切片の出力
print( '切片:', model.intercept_ )

# 決定係数の出力
print( '決定係数:', model.score(X, Y) )

## モデルの評価

In [ ]:
### モデルの可視化 ###

# 元データを散布図でプロット
plt.scatter(X, Y)

# 回帰直線をプロット
Y_pred = model.predict(X)             # Xをモデル式に代入して予測結果を算出
plt.plot(X['Area'], Y_pred, color='orange') # Xと予測結果Y_predをプロット

# プロット結果を表示
plt.show() 


In [ ]:
### 残差プロットの表示 ###

# 残差をプロット
plt.scatter(X, Y-Y_pred) # 残差（モデル予測結果Y_predと実測値Yとの差分）をプロット

# 補助線（残差=0）を引く
plt.hlines( y=0, xmin = 0, xmax = 300, color = 'black') # 見やすくするために、y=0の横一直線を太く表示

# グラフタイトルなどの設定（無くてもOK）
plt.title('残差プロット')                            # 図のタイトル
plt.xlabel('占有面積')                               # x軸のラベル
plt.ylabel('残差')                                   # y軸のラベル
plt.grid()                                           # グリッド線を表示

# プロット結果を表示
plt.show()

## データの準備（再）

### 外れ値を除去したい場合

In [ ]:
### 箱ひげ図による外れ値の存在確認 ###
df.boxplot()

In [ ]:
### 外れ値のチェック ###
df[ df['Price']>=17500 ]   # df[ 条件式 ] で条件式にマッチするレコードのみ抽出

In [ ]:
### 外れ値の除外方法 ###
df = df.drop(index=127) # df.drop(index=xxx) で、インデックス番号に合致するレコードを除外

In [ ]:
### 箱ひげ図による外れ値の除外結果確認 ###
df.boxplot()

## モデルの構築（再）

In [ ]:
### 単回帰分析 ###
from sklearn import linear_model # 機械学習パッケージに含まれた回帰分析モジュール"linear_model"を読込

# 説明変数のセット
X = df[ ['Area'] ]

# 目的変数のセット
Y = df['Price']

# 単回帰モデルを作成
model = linear_model.LinearRegression()  # インスタンス化（関数を使える状態にする）
model.fit(X, Y)                          # モデル構築（フィッティング）

# 偏回帰係数の出力
display( pd.DataFrame({"Variables":X.columns, "Coefficients":model.coef_}) )

# 切片の出力
print( '切片:', model.intercept_ )

# 決定係数の出力
print( '決定係数:', model.score(X, Y) )

## モデルの評価（再）

In [ ]:
### モデルの可視化 ###

# 元データを散布図でプロット
plt.scatter(X, Y)

# 回帰直線をプロット
Y_pred = model.predict(X)             # Xをモデル式に代入して予測結果を算出
plt.plot(X['Area'], Y_pred, color='orange') # Xと予測結果Y_predをプロット

# プロット結果を表示
plt.show() 


In [ ]:
### 残差プロットの表示 ###

# 残差をプロット
plt.scatter(X, Y-Y_pred) # 残差（モデル予測結果Y_predと実測値Yとの差分）をプロット

# 補助線（残差=0）を引く
plt.hlines( y=0, xmin = 0, xmax = 150, color = 'black') # 見やすくするために、y=0の横一直線を太く表示

# グラフタイトルなどの設定（無くてもOK）
plt.title('残差プロット')                            # 図のタイトル
plt.xlabel('占有面積')                               # x軸のラベル
plt.ylabel('残差')                                   # y軸のラベル
plt.grid()                                           # グリッド線を表示

# プロット結果を表示
plt.show()

# 演習3-2 重回帰分析

## モデルの構築

In [ ]:
### 重回帰分析 ###
from sklearn import linear_model # 機械学習パッケージに含まれた回帰分析モジュール"linear_model"を読込

# 説明変数のセット
X = df[ ['Minutes', 'Age', 'Area', 'One room'] ]

# 目的変数のセット
Y = df['Price']

# 重回帰モデルを作成	
model = linear_model.LinearRegression()  # インスタンス化（関数を使える状態にする）
model.fit(X, Y)      # モデル構築（フィッティング）

# 偏回帰係数の出力
display( pd.DataFrame({"Variables":X.columns, "Coefficients":model.coef_}) )

# 切片の出力
print('切片:', model.intercept_)

# 決定係数の出力
print('決定係数:', model.score(X, Y))

## モデルの評価

In [ ]:
### モデルの可視化 ###

# Xをモデル式に代入して予測結果を算出
Y_pred = model.predict(X)

# 「徒歩分数」の実測値と予測結果をプロット
plt.subplot(2, 2, 1)                               # plt.subplot(行数, 列数, 何番目のプロットか)で複数グラフを同時に描画可能
dataname = 'Minutes'
plt.scatter(X[dataname], Y)
# 回帰直線をプロット
plt.scatter(X[dataname], Y_pred, color = 'orange') # Xと予測結果Y_predをプロット
plt.xlabel(dataname)                               # X軸ラベル


# 「築年数」の実測値と予測結果をプロット
plt.subplot(2, 2, 2)                               # plt.subplot(行数, 列数, 何番目のプロットか)で複数グラフを同時に描画可能
dataname = 'Age'
plt.scatter(X[dataname], Y)
# 回帰直線をプロット
plt.scatter(X[dataname], Y_pred, color = 'orange') # Xと予測結果Y_predをプロット
plt.xlabel(dataname)                               # X軸ラベル


# 「占有面積」の実測値と予測結果をプロット
plt.subplot(2, 2, 3)                               # plt.subplot(行数, 列数, 何番目のプロットか)で複数グラフを同時に描画可能
dataname = 'Area'
plt.scatter(X[dataname], Y)
# 回帰直線をプロット
plt.scatter(X[dataname], Y_pred, color = 'orange') # Xと予測結果Y_predをプロット
plt.xlabel(dataname)                               # X軸ラベル


# 「ワンルーム」の実測値と予測結果をプロット
plt.subplot(2, 2, 4)                               # plt.subplot(行数, 列数, 何番目のプロットか)で複数グラフを同時に描画可能
dataname = 'One room'
plt.scatter(X[dataname], Y)
# 回帰直線をプロット
plt.scatter(X[dataname], Y_pred, color = 'orange') # Xと予測結果Y_predをプロット
plt.xlabel(dataname)                               # X軸ラベル


# グラフ同士が重ならないように設定
plt.tight_layout()                                 
    
# プロット結果を表示
plt.show() 


# 演習3-3 標準化（Zスコア）した重回帰分析

In [ ]:
from sklearn import preprocessing      # 機械学習パッケージscikit-learnから、データ前処理モジュールの読込

## データの準備

In [ ]:
### 標準化 ###
# 説明変数の標準化（Zスコア）
X_standard = preprocessing.scale(X)

# 目的変数の標準化（Zスコア）
Y_standard = preprocessing.scale(Y)

## モデルの構築

In [ ]:
### モデル構築（フィッティング） ###
model.fit(X_standard, Y_standard)

# 偏回帰係数の出力
display( pd.DataFrame({"Variables":X.columns, "Coefficients":model.coef_}) )

# 切片の出力
print('切片:', model.intercept_)

# 決定係数の出力
print('決定係数:', model.score(X_standard, Y_standard))

## 参考：モデルのチューニング（説明変数の除外）

In [ ]:
### 「ワンルーム」の除外 ###
X = X[ ['Minutes', 'Age', 'Area'] ]

In [ ]:
### 標準化 ###
# 説明変数の標準化（Zスコア）
X_standard = preprocessing.scale(X)

# 目的変数の標準化（Zスコア）
Y_standard = preprocessing.scale(Y)

In [ ]:
### モデル構築（フィッティング） ###
model.fit(X_standard, Y_standard)

# 偏回帰係数の出力
display( pd.DataFrame({"Variables":X.columns, "Coefficients":model.coef_}) )

# 切片の出力
print('切片:', model.intercept_)

# 決定係数の出力
print('決定係数:', model.score(X_standard, Y_standard))

# 演習3-4 ニューラルネットワークによる数値予測

## データの準備

In [ ]:
# 説明変数の再セット
X = df[ ['Minutes', 'Age', 'Area', 'One room'] ]

# 目的変数の再セット
Y = df['Price']

In [ ]:
### 標準化 ###
# 説明変数の標準化（Zスコア）
X_standard = preprocessing.scale(X)

# 目的変数の標準化（Zスコア）
Y_standard = preprocessing.scale(Y)

## モデルの構築

In [ ]:
# ニューラルネットワークのライブラリ読み込み
from sklearn.neural_network import MLPRegressor #scikit-learnパッケージ内の ニューラルネットワークのモジュールより、
                                                #多層パーセプトロン (MLP) 回帰のクラスを読込

# ニューラルネットワークモデル構築
model = MLPRegressor( random_state=0, hidden_layer_sizes=(100,))   #インスタンス化（関数を使える状態にする）
            #パラメータの説明：
            # random_state: 乱数のシード値。指定することで再現性を担保
            # hidden_layer_sizes=(100,) は隠れ層1、ノード数100を意味する

model.fit(X_standard, Y_standard)  #モデル構築（フィッティング）

## モデルの評価

In [ ]:
# モデル予測値の出力
Y_pred = model.predict(X_standard)

# 精度の出力
model.score(X_standard, Y_standard)

In [ ]:
#参考：各種パラメータのデフォルト値
# MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
#        beta_2=0.999, early_stopping=False, epsilon=1e-08,
#        hidden_layer_sizes=(100,), learning_rate='constant',
#        learning_rate_init=0.001, max_iter=200, momentum=0.9,
#        nesterovs_momentum=True, power_t=0.5, random_state=None,
#        shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
#        verbose=False, warm_start=False)

# 参考：分布の対数変換

In [ ]:
# 対数変換した上でヒストグラム表示
df['Price'].apply(np.log).hist() 
        # np.logで対数変換
        #  ※データに0を含む場合はエラーが出るので、事前に0の除外が必要
        #     e.g. df[ df['Age']!=0 ]['Age']  
        # df[特定カラム].apply(xxx)で網羅的に各レコードに適用
